In [153]:
import nltk
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk import ngrams 
import math

def remove_stopwords_and_punctuation(text_data):
    tokens = nltk.word_tokenize(text_data)
    token_punc_removed = list(filter(lambda word: word not in "\//:@()/!#'&,--.';``''", tokens))
    #token_stopwords_removed = list(filter(lambda word: word not in set(stopwords.words('english')), token_punc_removed))
    token_stopwords_removed = [word for word in token_punc_removed if word not in stopwords.words('english')]
    norm_tokens = [word.lower() for word in token_stopwords_removed]
    return " ".join(norm_tokens)

def count_words(text_data):
    k = 0
    w = word_tokenize(text_data)
    for word in w:
        k += 1
    return k

def wrd_frq_dict(tweet,cutoff = 0):
    i = 0
    freq_dict = {}
    words = word_tokenize(tweet)
    for word in words:
        if tweet.count(word) > cutoff: freq_dict[word] = tweet.count(word)
    return freq_dict

def pre_process_tweets(tweets):
    pre_processed_tweets = []
    for tweet in tweets:    
        cleaned_tweet = remove_stopwords_and_punctuation(tweet)
        cleaned_tweet = cleaned_tweet.replace(' the ', '').replace("'s",'').replace('a ','')
        pre_processed_tweets.append(cleaned_tweet.split())
    return pre_processed_tweets


In [128]:
file_handle = open('storm_tweets.txt') # text file containing more than 200 words
storm_tweets = file_handle.read().splitlines()
storm_tweets = list(filter(None, storm_tweets))


In [138]:
tweets_frq_dict = {}
pre_processed_tweets = pre_process_tweets(storm_tweets)
tweet = sum(pre_processed_tweets,[])
tweets_frq_dict = wrd_frq_dict(' '.join(tweet))
tweets_frq_dict.keys()

dict_keys(['stormcallum', 'coming', 'thursday', 'night', 'friday', 'morning', 'severe', 'gales', 'especially', 'coastal', 'counties', 'high', 'tides', 'could', 'cause', 'flooding', 'gusts', '130km/h', '80mph', 'exposed', 'areas', 'breaking', 'weather', 'officially', 'named', 'met', 'eireann', 'already', 'carries', 'status', 'orange', 'warning', 'level', 'western', 'southern', 'eastern', 'here', 'latest', 'outlook', 'stay', 'tuned', 'updates', 'system', 'meteireannstrongest', 'winds', 'likely', 'ireland', 'though', 'nw', 'parts', 'uk', 'expect', 'galesmain', 'impacts', 'heavy', 'rain', 'across', 'saturday', 'weatheraware', 'updatelatest', 'ecmwf', 'forecast', 'model', 'shifted', 'storm', 'track', 'east', 'indicating', 'greater', 'swathe', 'country', 'risk', 'seeing', 'damaging', 'excess', '160', 'km/h', 'seem', 'run', 'shifts', 'center', 'west', 'keeping', 'strongest', 'offshore', 'still', 'bringing', 'strong', 'intense', 'rainfall', 'many', 'hopefully', 'may', 'become', 'trend', 'move'

In [148]:
terms_bigram = []
total_words_of_alltweets = 0
tweet = tweets_frq_dict.keys()
terms_bigram.append(list(ngrams(tweet, 2)))
total_words_of_alltweets = len(tweet)
    
single_bigrams_list = sum(terms_bigram, [])
bigram_frq_dict = {}
cutoff = 0

for tweet_bigram in single_bigrams_list:
    if single_bigrams_list.count(tweet_bigram) > cutoff:
        bigram_frq_dict[tweet_bigram] = single_bigrams_list.count(tweet_bigram)
    
p_word = {}
for term, n in tweets_frq_dict.items():
    p_word[term] = n / total_words_of_alltweets

p_bigram = {}
for tuple_bigram,n in bigram_frq_dict.items():
    p_bigram[tuple_bigram] =  n/total_words_of_alltweets
        


In [149]:
pmi = {}
for t1t2,pt1t2 in p_bigram.items():
    t1 = t1t2[0]
    t2 = t1t2[1]
    pmi[t1t2] = math.log(pt1t2/float(p_word[t1]*p_word[t2]),2)

pmi

{('10pm', 'valid'): 7.459431618637297,
 ('130km/h', '80mph'): 6.459431618637298,
 ('160', 'km/h'): 6.459431618637298,
 ('2', 'moderate'): 6.459431618637298,
 ('58', 'hpa/24h'): 7.459431618637297,
 ('70mph', 'there'): 7.459431618637297,
 ('80mph', 'exposed'): 6.459431618637298,
 ('90mph', 'cat4'): 7.459431618637297,
 ('across', 'saturday'): 7.459431618637297,
 ('advanced', 'warnings'): 7.459431618637297,
 ('ahead', 'arrival'): 7.459431618637297,
 ('already', 'carries'): 6.459431618637298,
 ('areas', 'breaking'): 5.874469117916141,
 ('arrival', 'callum'): 4.137503523749936,
 ('atlantic', 'developing'): 6.459431618637298,
 ('become', 'trend'): 7.459431618637297,
 ('blustery', 'my'): 7.459431618637297,
 ('breaking', 'weather'): 5.459431618637297,
 ('bringing', 'strong'): 5.137503523749935,
 ('callum', 'weekwarning'): 4.137503523749936,
 ('canadpower', 'carried'): 7.459431618637297,
 ('captured', 'presence'): 7.459431618637297,
 ('carried', 'jet'): 6.459431618637298,
 ('carries', 'status'):

In [152]:
#tuple list conversion
pmi_pairstuple_list = []
for t1,t2 in pmi.items():
    pmi_pairstuple_list.append((t1, t2))
pmi_ordered = sorted(pmi_pairstuple_list, key=operator.itemgetter(1), reverse=True)
pmi_ordered[10:20]

[(('nw', 'parts'), 7.459431618637297),
 (('expect', 'galesmain'), 7.459431618637297),
 (('galesmain', 'impacts'), 7.459431618637297),
 (('impacts', 'heavy'), 7.459431618637297),
 (('across', 'saturday'), 7.459431618637297),
 (('saturday', 'weatheraware'), 7.459431618637297),
 (('weatheraware', 'updatelatest'), 7.459431618637297),
 (('indicating', 'greater'), 7.459431618637297),
 (('greater', 'swathe'), 7.459431618637297),
 (('swathe', 'country'), 7.459431618637297)]